In [38]:
from pyspark.sql import SparkSession

In [39]:
spark = SparkSession \
    .builder \
    .appName("SparkSQL") \
    .getOrCreate()

path = "data/sparkify_log_small.json"
table= spark.read.json(path)

In [40]:
# Created table view
table.createOrReplaceTempView("tableview")

In [41]:
# Simple SQL query
spark.sql("SELECT * FROM tableview LIMIT 2").show()

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|   Lily Allen|Logged In|Elizabeth|     F|            7|   Chase|195.23873| free|Shreveport-Bossie...|   PUT|NextSong|1512718541284|     5027|      

In [42]:
#UserDefined Functions
## Register the udf
spark.udf.register("get_hour", lambda x: int(datetime.datetime.fromtimestamp(x / 1000.0).hour))

<function __main__.<lambda>(x)>

In [43]:
songs_in_hour = spark.sql('''
          SELECT get_hour(ts) AS hour, COUNT(*) as plays_per_hour
          FROM tableview
          WHERE page = "NextSong"
          GROUP BY hour
          ORDER BY cast(hour as int) ASC
          '''
          )

In [53]:
#Which page did user id ""(empty string) NOT visit?
query1 = '''
        SELECT DISTINCT page
        from tableview
        where page not in
        (SELECT DISTINCT page FROM tableview
        WHERE userId == "")
        '''       

In [59]:
#How many female users do we have in the data set?
query2 = '''
         SELECT count(DISTINCT userId) 
         from tableview
         where gender=='F'
         
         '''

In [68]:
#How many songs were played from the most played artist?
query3 = '''
        SELECT count('song') as new_count
        from tableview
        where page ="NextSong"
        group by artist
        order by new_count desc
        LIMIT 1
         '''

In [184]:
#How many songs do users listen to on average between visiting our home page? 
#Please round your answer to the closest integer.
query4 = '''
         WITH t AS (SELECT userID, page, ts, CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS home_flag FROM tableview \
            WHERE (page = 'NextSong') or (page = 'Home')),
         
         t1 AS (SELECT *, SUM(home_flag) OVER \
               (PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period \
                FROM t
         ),
         
         t2 AS (SELECT COUNT(*) AS count_results FROM t1 
                WHERE page = 'NextSong'
                GROUP BY userID, period, page
         )
         
          SELECT AVG(count_results) as average FROM t2
         '''

In [185]:
ans = spark.sql(query4)

In [186]:
ans.show()

+-----------------+
|          average|
+-----------------+
|6.898347107438017|
+-----------------+

